In [ ]:
# print ('running part 2')
# print ()
# !python part2.py

# print ()

# print ('running part 3')
# print ()
# !python part3.py

# print ()

# print ('running part 4')
# print ()
# !python part4.py

# print ()

print ('running part 5')
print ()
!python part5.py


# my_dict = {'hello':0, 'my':1, 'name':2, 'is:':3, 'Reuben':4}

# # print (max(my_dict.values()))


running part 5

